In [18]:
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
#nltk.download('punkt_tab')
import numpy as np
from transformers import BartTokenizer, BartForConditionalGeneration

from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama




In [2]:
# get the link
link = "https://www.youtube.com/watch?v=JSTUtRQ8Hwc" 
# get the unique ID
unique_id = link.split("=")[-1]
# get the transcript of the video based on the unique ID
sub = YouTubeTranscriptApi.get_transcript(unique_id)  
# get only the "text" parts and join them together, ignoring others like "duration"
subtitle = " ".join([x['text'] for x in sub])

In [7]:
print(subtitle)
#print(repr(subtitle))  # This will display the raw text with all special characters

I have five side projects that will give you an unfair Advantage at getting a job it's not another to-do list app or some Netflix clone it's an actual product for real paying customers so here's the deal if you build one of these products I will be your first paying customer I'm also looking for a founding engineer at my new company so if you build one of these projects I'll automatically consider you for an engineering role at my company if you'd like building a real product for real customers carries way more weight than you know whatever School you went to I'll give you some rules later on but let's just get into the ideas idea number one give GitHub productivity vs code extension let's talk about what everyone is most embarrassed about their GitHub activity monitor the thing basically shows how much you've coded in the last year people use it as a badge of honor to show how much they code go off there's even a side project that went viral on Twitter called ship talkers where they w

In [ ]:
# 1
def summarise(text):

    PROMPT_TEMPLATE = """
    You are a helpful AI assistent, Please provide a concise summary of the following text, highlighting the main points and key details while keeping the essence of the content intact. Here’s the text: 
    {text}
    summarise the important """

    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)  
    prompt = prompt_template.format(text=subtitle)

    model = Ollama(model="llama3.2")  
    summarised_text = model.invoke(prompt)
    return summarised_text


In [14]:
# 2
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# encode the subtitle using Bart tokenizer
input_tensor = tokenizer.encode( subtitle, return_tensors="pt", max_length=512)

outputs_tensor = model.generate(input_tensor, max_length=1024, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
#outputs_tensor

print(tokenizer.decode(outputs_tensor[0]))


In [56]:
# 3
def summarize1(link):
    unique_id = link.split("=")[-1]
    sub = YouTubeTranscriptApi.get_transcript(unique_id)
    subtitle = " ".join([x['text'] for x in sub])
    subtitle = subtitle.replace("\\", "")
    sentences = sent_tokenize(subtitle)
    organized_sent = {k:v for v,k in enumerate(sentences)}
    tf_idf = TfidfVectorizer(min_df=2, 
                                    strip_accents='unicode',
                                    max_features=None,
                                    lowercase = True,
                                    token_pattern=r'w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=True,
                                    smooth_idf=True,
                                    sublinear_tf=True,
                                    stop_words = 'english')

    sentence_vectors = tf_idf.fit_transform(sentences)
    sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()
    N = 3
    top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]
    mapped_sentences = [(sentence,organized_sent[sentence]) for sentence in top_n_sentences]
    mapped_sentences = sorted(mapped_sentences, key = lambda x: x[1])
    ordered_sentences = [element[0] for element in mapped_sentences]
    summary = " ".join(ordered_sentences)
    return summary
    